In [92]:
import os
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer
from sklearn import metrics
import mlflow
import pickle
from hyperopt import hp
from hyperopt.pyll import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [93]:
def read_dataframe(filepath):
    df = pd.read_csv(filepath)
    return df

In [94]:
def preparing_dataframe(filepath):
    df = read_dataframe(filepath)
    
    y_column = "active_customer"
    train_columns = ["customer_age",
                     "gender",
                     "dependent_count",
                     "education_level",
                     "marital_status",
                     "income_category",
                     "card_category",
                     "months_on_book",
                     "total_relationship_count",
                     "credit_limit",
                     "total_revolving_bal"]
    df_to_split = df[train_columns+[y_column]]
    return df_to_split , y_column


In [95]:
def split_dataFrame(filepath):
    
    df_to_split , y_column = preparing_dataframe(filepath)
    
    df_full_train, df_test = train_test_split(df_to_split, test_size=0.2, random_state=11)
    df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)

    df_full_train = df_full_train.reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    y_full_train = df_full_train[y_column]
    y_train = df_train[y_column]
    y_val = df_val[y_column]
    y_test = df_test[y_column]

    del df_full_train[y_column]
    del df_train[y_column]
    del df_val[y_column]
    del df_test[y_column]

    # with pd.option_context('display.max_rows', 2, 'display.max_columns', None):
    #     display(df_test)

    #print("df_to_split length: ", len(df_to_split))
    #print()
    #print("df_full_train length: ", len(df_full_train))
    #print("df_train length: ", len(df_train))
    #print("df_val length: ", len(df_val))
    #print("df_test length: ", len(df_test))
    #print()
    #print("y_full_train length: ", len(y_full_train))
    #print("y_train length: ", len(y_train))
    #print("y_val length: ", len(y_val))
    #print("y_test length: ", len(y_test))

    return df_full_train, df_train, df_val, df_test, y_full_train, y_train, y_val, y_test


In [96]:
def train(dataFrame, y, xgb_params):
    # Hot Encoding
    dicts = dataFrame.to_dict(orient="records")
    dv = DictVectorizer(sparse=False)
    X = dv.fit_transform(dicts)

    features = dv.get_feature_names_out()
    #print(features)
    dtrain = xgb.DMatrix(X, label=y, feature_names=features, enable_categorical=True)

    # train
    model = xgb.train(xgb_params, dtrain, num_boost_round=10)
    #print(model.feature_names)

    return dv, model


In [97]:
def predict(dataFrame, dv, model):
    dicts = dataFrame.to_dict(orient="records")
    X = dv.transform(dicts)
    features = dv.get_feature_names_out()
    dval = xgb.DMatrix(X, feature_names=features)
    y_pred = model.predict(dval)
    return y_pred, X

In [98]:
def get_rmse(y_val, y_pred_val):
    mae = metrics.mean_absolute_error(y_val, y_pred_val)
    mse = metrics.mean_squared_error(y_val, y_pred_val)
    rmse = np.sqrt(metrics.mean_squared_error(y_val, y_pred_val))

    #print("MAE for numerical linear:", mae)
    #print("MSE for numerical linear:", mse)
    #print("RMSE:", rmse)
    return mae, mse, rmse

In [99]:
def set_mlflow(experiment):
    mlflow.set_tracking_uri("http://15.207.72.49:5000")
    mlflow.set_experiment(experiment)

In [100]:
def run(experiement, filepath):
    
    #df_clean = read_dataframe(filepath)
    #df_to_split , y_column = preparing_dataframe(filepath)
    df_full_train, df_train, df_val, df_test, y_full_train, y_train, y_val, y_test = split_dataFrame(filepath)

    mlflow.xgboost.autolog()

    xgb_params_search_space = {
        'max_depth': scope.int(hp.choice('max_depth', [5, 10, 12, 13, 14, 20,30,40,50,100])),
        'eta': scope.int(hp.choice('eta', [0, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.6, 1, 2, 5, 10])),
        'min_child_weight': 1,
        'objective': 'reg:squarederror',
        'nthread': 8,
        'verbosity':0,
        "seed":42
    }

    def objective(params={}):
        with mlflow.start_run():
            active_mlflow_run_id = mlflow.active_run().info.run_id
            if (active_mlflow_run_id==None): raise ValueError("missing MLFlow active run.")
            # print(f'Training model. Active MLFlow run_id: {active_mlflow_run_id}')
            mlflow.set_tag("model", "xgboost")

            dv, model = train(df_full_train, y_full_train, params)
            
            os.makedirs("models", exist_ok=True)
            with open("models/preprocessor.b","wb") as f_out:
                pickle.dump(dv,f_out)
            mlflow.log_artifact("models/preprocessor.b",artifact_path="preprocessor")
            mlflow.xgboost.log_model(model,artifact_path="models_mlflow")
            print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

            y_pred_val, X_val = predict(df_val, dv, model)
            mae, mse, rmse = get_rmse(y_val, y_pred_val)
            mlflow.log_metric("mae", mae) 
            mlflow.log_metric("mse", mse) 
            mlflow.log_metric("rmse", rmse) 

        return {'loss':rmse, 'status':STATUS_OK}

    best_result = fmin(
        fn=objective,
        space=xgb_params_search_space,
        algo=tpe.suggest,
        max_evals=5, #50
        trials=Trials()
    )

In [101]:
def register_best_run(experiment_name):
    client = MlflowClient()

    # select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(experiment_name)
    print(experiment)
    best_runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=5,  #50
        order_by=["metrics.rmse ASC"]
    )

    print(f'Models count: {len(best_runs)}')
    if (len(best_runs) == 0): raise "No models found."
    print(f'Top model found: {best_runs[0]}')

    # register the best model
    model_uri = f"runs:/{best_runs[0].info.run_id}/model"
    print(f'Registering {model_uri}')
    mv = mlflow.register_model(model_uri=model_uri, name=f"best_model-{experiment_name}")
    print(f"Registrered model {mv.name}, version: {mv.version}")
    # client.update_registered_model(
    #     name=mv.name,
    #     description=f"rmse={best_runs[0].data.metrics['rmse']}"
    # )
    # client.list_registered_models()

In [102]:
if __name__=="__main__":
    
    filepath = "../Clean_Input_Data/credit_card_churn_clean.csv"
    
    experiment = input("Enter Experiment name : ")
    
    set_mlflow(experiment)
    run(experiment, filepath)
    register_best_run(experiment)
    
    
    
    
    

Enter Experiment name : exp-7


2023/02/02 03:35:44 INFO mlflow.tracking.fluent: Experiment with name 'exp-7' does not exist. Creating a new experiment.
2023/02/02 03:35:45 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


  0%|                                                                                                                | 0/5 [00:00<?, ?trial/s, best loss=?]

2023/02/02 03:35:47 WARNING mlflow.xgboost: Failed to infer model signature: training data did not have the following fields: card_category=blue, card_category=gold, card_category=platinum, card_category=silver, credit_limit, customer_age, dependent_count, education_level, gender=F, gender=M, income_category, marital_status=divorced, marital_status=married, marital_status=single, months_on_book, total_relationship_count, total_revolving_bal



default artifacts URI: 's3://mlops-s3-bucket-1/11/e712d98cbbce4ba7a175f3f4b4a71b1a/artifacts'                                                              
 20%|████████████████████▏                                                                                | 1/5 [00:50<03:20, 50.23s/trial, best loss: 0.5]

...

2023/02/02 03:37:09 WARNING mlflow.xgboost: Failed to infer model signature: training data did not have the following fields: card_category=blue, card_category=gold, card_category=platinum, card_category=silver, credit_limit, customer_age, dependent_count, education_level, gender=F, gender=M, income_category, marital_status=divorced, marital_status=married, marital_status=single, months_on_book, total_relationship_count, total_revolving_bal



default artifacts URI: 's3://mlops-s3-bucket-1/11/6117480acd0a438295dabfb24f6f744f/artifacts'                                                              
100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:32<00:00, 18.47s/trial, best loss: 0.0006259383517317474]
<Experiment: artifact_location='s3://mlops-s3-bucket-1/11', creation_time=1675289144223, experiment_id='11', last_update_time=1675289144223, lifecycle_stage='active', name='exp-7', tags={}>
Models count: 5
Top model found: <Run: data=<RunData: metrics={'mae': 0.0003868605417665094,
 'mse': 3.9179880673145817e-07,
 'rmse': 0.0006259383517317474}, params={'custom_metric': 'None',
 'early_stopping_rounds': 'None',
 'eta': '1',
 'max_depth': '100',
 'maximize': 'None',
 'min_child_weight': '1',
 'nthread': '8',
 'num_boost_round': '10',
 'objective': 'reg:squarederror',
 'seed': '42',
 'verbose_eval': 'True',
 'verbosity': '0'}, tags={'mlflow.log-model.history': '[{"run_id": "96bd8e5

Successfully registered model 'best_model-exp-7'.
2023/02/02 03:37:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best_model-exp-7, version 1


Registrered model best_model-exp-7, version: 1


Created version '1' of model 'best_model-exp-7'.
